# Tarea 2 — Inteligencia Artificial (UDP)

**Instrucciones clave (resumen de la pauta):**  
- **Clustering**: K-Means, K-Means++ y MeanShift, **al menos 4 configs por técnica**, entrenar con **80%** (sin Y), evaluar con métrica (p.ej., Silhouette) y **elegir top-3**; aplicar al **20%**, asignar etiqueta por **clase dominante del cluster** y discutir razonabilidad.  
- **Supervisado**: Entrenar en **paralelo** múltiples instancias (≥3 por técnica) de **Regresión Logística** y **SVM**, **variando hiperparámetros** definidos en un **archivo de configuración externo**; entrenar con **80%** y **cada 5 épocas eliminar la peor**; para las **mejores 2 configs**, reportar métricas en **20%**.  

> **Nota académica:** Esta notebook contiene **código y estructura**. Debes **redactar tu propio análisis** en las celdas marcadas como *[Completar análisis]*. 


## 0) Setup del entorno (local)

1. Asegúrate de tener un entorno de Python 3.10+ (puede ser `conda` o `venv`).  
2. Instala dependencias en tu máquina:
```bash
pip install -r ../requirements.txt
```
Si usas VS Code, puedes abrir esta carpeta y ejecutar esta notebook con la extensión de Jupyter.


In [63]:
# Imports principales
import os, json, yaml
import numpy as np
import pandas as pd
import sys

from pathlib import Path
from sklearn.metrics import silhouette_score

ROOT = Path.cwd().parent   # notebooks/ -> (padre) = raíz del proyecto
sys.path.insert(0, str(ROOT))
print("Proyecto en PYTHONPATH:", ROOT)

# (opcional) aseguramos que 'src' sea paquete válido
(Path(ROOT/"src"/"__init__.py")).touch()

from src.utils import read_config, load_dataset, build_preprocessor, split_xy, make_splits
from src.clustering import run_clustering
from src.supervised import train_with_elimination

np.random.seed(42)


Proyecto en PYTHONPATH: /home/martuko/Universidad/IA/t2_ai_udp


## 1) Cargar configuración y dataset

- Edita `../config/config.yaml` para ajustar:
  - `dataset_path`: ruta a tu CSV (por ejemplo, `data/ai4i_2020.csv` descargado de Kaggle).
  - `target_column`: nombre exacto de la columna Y (p.ej., `Failure Type`).
  - `drop_columns`, `categorical_columns`: ajusta de acuerdo a tu archivo.


In [64]:
from pathlib import Path
import sys, importlib

ROOT = Path.cwd().parent  # .../t2_ai_udp
sys.path.insert(0, str(ROOT))

import src.utils as utils
importlib.reload(utils)

from src.utils import read_config, load_dataset, build_preprocessor, split_xy, make_splits
cfg = read_config("../config/config.yaml")
cfg


{'dataset_path': '../data/ai4i2020.csv',
 'target_column': 'Failure Type',
 'drop_columns': ['UDI',
  'Product ID',
  'Machine failure',
  'TWF',
  'HDF',
  'PWF',
  'OSF',
  'RNF'],
 'categorical_columns': ['Type'],
 'test_size': 0.2,
 'random_state': 42,
 'clustering': {'score_metric': 'silhouette',
  'kmeans': {'configs': [{'name': 'km_rand_k2',
     'n_clusters': 2,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_rand_k3',
     'n_clusters': 3,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_rand_k5',
     'n_clusters': 5,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_rand_k8',
     'n_clusters': 8,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_pp_k3',
     'n_clusters': 3,
     'init': 'k-means++',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_pp_k4',
     'n_clusters': 4,
     'init': 'k-means++',
     'n_init': 20,
     'max_ite

In [65]:
# Cargar CSV y chequear columnas
df, target_col, num_cols, cat_cols = load_dataset(cfg)
print(f"Shape: {df.shape}")
print("Columnas:", df.columns.tolist())
print("Objetivo (Y):", target_col)
print("Categóricas detectadas:", cat_cols)
print("Numéricas:", num_cols)

# Resumen de clases (para verificar multiclase)
print("\nDistribución de la etiqueta Y:")
print(df[target_col].value_counts(dropna=False))


Shape: (10000, 7)
Columnas: ['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Failure Type']
Objetivo (Y): Failure Type
Categóricas detectadas: ['Type']
Numéricas: ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

Distribución de la etiqueta Y:
Failure Type
No Failure                  9652
Heat Dissipation Failure     115
Power Failure                 91
Overstrain Failure            78
Tool Wear Failure             46
Random Failures               18
Name: count, dtype: int64


## 2) Separar train/test (80/20) y preparar *preprocessor*


In [66]:
X, y = split_xy(df, target_col)
X_train, X_test, y_train, y_test = make_splits(X, y, test_size=cfg['test_size'], random_state=cfg['random_state'])
print(X_train.shape, X_test.shape)

# El preprocesador se ajusta solo con train (para evitar leakage)
pre = build_preprocessor(num_cols, cat_cols)
Xtr = pre.fit_transform(X_train)
Xte = pre.transform(X_test)
Xtr.shape, Xte.shape


(8000, 6) (2000, 6)


((8000, 8), (2000, 8))

## 3) Clustering: K-Means / K-Means++ / MeanShift

- Entrenamos **solo con X_train** (sin Y), probando ≥4 configuraciones por técnica.
- Evaluamos con la métrica elegida (p.ej. *Silhouette*). Elegimos el **Top-3** y medimos la **asignación por etiqueta dominante** en el 20% de test.


In [67]:
df_results, top3, pre_used = run_clustering(cfg, X_train, y_train, X_test, y_test, num_cols, cat_cols)
print("== Resultados (ordenado por métrica en train) ==")
display(df_results.head(12))
print("\n== TOP-3 por métrica en train ==")
display(top3)


== Resultados (ordenado por métrica en train) ==


,technique,name,n_clusters,params,score_metric,train_score,test_dominant_label_acc,bandwidth
0,MeanShift,ms_q_0.10,NaN,"{'name': 'ms_q_0.10', 'quantile': 0.1, 'n_samp...",silhouette,0.389936,0.966,1.895192
1,KMeans,km_rand_k2,2.0,"{'name': 'km_rand_k2', 'n_clusters': 2, 'init'...",silhouette,0.229119,0.965,NaN
2,KMeans++,km_pp_k3,3.0,"{'name': 'km_pp_k3', 'n_clusters': 3, 'init': ...",silhouette,0.228268,0.965,NaN
3,KMeans,km_rand_k3,3.0,"{'name': 'km_rand_k3', 'n_clusters': 3, 'init'...",silhouette,0.228267,0.965,NaN
4,KMeans++,km_pp_k4,4.0,"{'name': 'km_pp_k4', 'n_clusters': 4, 'init': ...",silhouette,0.202251,0.965,NaN
5,KMeans++,km_pp_k5,5.0,"{'name': 'km_pp_k5', 'n_clusters': 5, 'init': ...",silhouette,0.188518,0.965,NaN
6,KMeans,km_rand_k5,5.0,"{'name': 'km_rand_k5', 'n_clusters': 5, 'init'...",silhouette,0.188507,0.965,NaN
7,KMeans++,km_pp_k6,6.0,"{'name': 'km_pp_k6', 'n_clusters': 6, 'init': ...",silhouette,0.183999,0.965,NaN
8,KMeans,km_rand_k8,8.0,"{'name': 'km_rand_k8', 'n_clusters': 8, 'init'...",silhouette,0.177370,0.965,NaN
9,MeanShift,ms_bw_1.0,NaN,"{'name': 'ms_bw_1.0', 'quantile': None, 'n_sam...",silhouette,0.108982,0.966,1.000000



== TOP-3 por métrica en train ==


,technique,name,n_clusters,params,score_metric,train_score,test_dominant_label_acc,bandwidth
0,MeanShift,ms_q_0.10,NaN,"{'name': 'ms_q_0.10', 'quantile': 0.1, 'n_samp...",silhouette,0.389936,0.966,1.895192
1,KMeans,km_rand_k2,2.0,"{'name': 'km_rand_k2', 'n_clusters': 2, 'init'...",silhouette,0.229119,0.965,NaN
2,KMeans++,km_pp_k3,3.0,"{'name': 'km_pp_k3', 'n_clusters': 3, 'init': ...",silhouette,0.228268,0.965,NaN


### [Completar análisis] — *Clustering*
- Comenta brevemente las configuraciones que obtuvieron mejor puntaje (métrica en train).
- Discute si la asignación de **etiqueta dominante por cluster** al 20% de test parece razonable para imputar etiquetas faltantes.
- Agrega gráficos si lo estimas necesario (p.ej., distribución de tamaños de cluster). **No uses IA generativa para redactar este análisis.**


## 4) Modelos Supervisados (Logistic / SVM) con eliminación periódica

- Entrenamos **en paralelo** múltiples configuraciones (definidas en `config.yaml`), **solo con train**.
- Cada **5 épocas**, eliminamos la **peor** configuración (según métrica en *train*).
- Reportamos métricas en **test** para las **dos mejores** configuraciones finales.


In [61]:
import importlib, src.supervised as sup
importlib.reload(sup)
from src.supervised import train_with_elimination


In [62]:
# (Opcional) Puedes ajustar n_jobs según tu CPU; -1 usa todos los núcleos
train_history, test_summary, survivors = train_with_elimination(cfg, Xtr, y_train, Xte, y_test, class_names=None, n_jobs=-1)

print("== Historial de evaluación (train) ==")
display(train_history)

print("\n== Métricas en test para las 2 mejores configuraciones ==")
display(test_summary[["name","algo","test_accuracy","test_f1_macro"]])

# Guardar resultados
Path("outputs").mkdir(exist_ok=True)
train_history.to_csv("outputs/train_history.csv", index=False)
test_summary.to_csv("outputs/test_summary.csv", index=False)
print("Archivos guardados en outputs/: train_history.csv y test_summary.csv")


== Historial de evaluación (train) ==


,epoch,name,algo,metric,score
0,5,svm_modhub_opt_a1e-4,svm,f1_macro,0.170505
1,5,log_optimal_a1e-4,logistic,f1_macro,0.163720
2,5,svm_hinge_opt_a1e-4,svm,f1_macro,0.163720
3,5,svm_hinge_inv_a1e-3,svm,f1_macro,0.163720
4,5,log_inv_a1e-3,logistic,f1_macro,0.135111
5,5,log_adapt_a5e-4,logistic,f1_macro,0.028752
6,5,log_optimal_a5e-5,logistic,f1_macro,0.003790
7,5,svm_hinge_adp_a5e-4,svm,f1_macro,0.003014
8,10,log_optimal_a1e-4,logistic,f1_macro,0.188886
9,10,svm_hinge_inv_a1e-3,svm,f1_macro,0.170564



== Métricas en test para las 2 mejores configuraciones ==


,name,algo,test_accuracy,test_f1_macro
0,log_optimal_a1e-4,logistic,0.9650,0.163907
1,svm_hinge_opt_a1e-4,svm,0.9665,0.205699


Archivos guardados en outputs/: train_history.csv y test_summary.csv


### [Completar análisis] — *Supervisado*
- Explica cómo afectaron los **hiperparámetros** al rendimiento (por ejemplo, `alpha`, `learning_rate`, `loss`).
- Compara **Regresión Logística** vs **SVM** en las métricas de test (accuracy, F1 macro).
- Comenta brevemente el efecto de la **eliminación cada 5 épocas** sobre la convergencia de las configuraciones.
